In [6]:
import polars as pl             # faster and more efficient than pandas
import pyarrow as pa
import pyarrow.parquet as pq
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
import urllib.parse
# base_url = 'https://data.cityofnewyork.us/api/v3/views/p937-wjvj/query.csv'
block_size = 100_000  # Number of rows per block
#output_parquet = 'rodent_inspection_data.parquet'
output_parquet = '311_service_requests_data.parquet'

base_url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv'
max_workers = 4 # Number of threads for parallel processing

columns = [ # channel_type --> open_data_channel_type
    "unique_key", "created_date", "closed_date", "agency", "agency_name",
    "complaint_type", "descriptor", "location_type", "incident_zip",
    "city", "status", "resolution_action_updated_date", "borough", "open_data_channel_type"
]

def download_block (offset):
    # Socrata API in SQL -- pending to test SODA 3
    soql = (
        f"SELECT {', '.join(columns)} "
        f"LIMIT {block_size} OFFSET {offset}"  
    )
    
    encoded_query = urllib.parse.quote(soql, safe='')
    url = f"{base_url}?$query={encoded_query}"
    
    try: 
        df_block = pl.read_csv(url, 
                               columns=columns, 
                               schema_overrides={"incident_zip": pl.Utf8}
                               )
        if df_block.height == 0:
            return None
        table = df_block.to_arrow() # File type transformation to arrow
        return (offset, table)
    except Exception as e:
        print(f"Error downloading block at offset {offset}: {e}")
        return None
    


In [ ]:
# download loop and offset counter

offset = 0
first_block = True
writer = None

while True:
    # Use ThreadPoolExecutor to download multiple blocks in parallel
    offsets = [offset + i*block_size for i in range(max_workers)]
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_block, o): o for o in offsets}
        
        all_done = False
        for future in as_completed(futures):
            result = future.result()
            if result is None:
                all_done = True
                continue
            
            block_offset, table = result
            
            # Initialize ParquetWriter on first block
            if first_block:
                writer = pq.ParquetWriter('311_service_requests_data.parquet', table.schema, compression='snappy')
                first_block = False
            
            writer.write_table(table)
            print(f"Downloaded and wrote block at offset {block_offset}")
    if all_done:
        break
    offset += max_workers * block_size
    
# Closing file writer
if writer:
    writer.close()

print("All data downloaded and saved to Parquet.")

In [9]:
# building dimensional models --> modified application, create single build

import urllib.parse
base_url = 'https://data.cityofnewyork.us/resource/p937-wjvj.csv'
block_size = 100_000  # Number of rows per block
output_parquet = 'rodent_inspection_data.parquet'
#output_parquet = '311_service_requests_data.parquet'

#base_url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv'
max_workers = 4 # Number of threads for parallel processing

columns = [ # inspection dimension
    "job_ticket_or_work_order_id", "job_id", "job_progress", "inspection_date", "result",
    "borough", "inspection_type", "zip_code" # added attributes for analysis
]

def download_block (offset):
    # Socrata API in SQL -- pending to test SODA 3
    soql = (
        f"SELECT {', '.join(columns)} "
        f"LIMIT {block_size} OFFSET {offset}"  
    )
    
    encoded_query = urllib.parse.quote(soql, safe='')
    url = f"{base_url}?$query={encoded_query}"
    
    try: 
        df_block = pl.read_csv(url, 
                               columns=columns, 
                               schema_overrides={"zip_code": pl.Utf8}
                               )
        if df_block.height == 0:
            return None
        table = df_block.to_arrow() # File type transformation to arrow
        return (offset, table)
    except Exception as e:
        print(f"Error downloading block at offset {offset}: {e}")
        return None
    

In [10]:
# download loop and offset counter

offset = 0
first_block = True
writer = None

while True:
    # Use ThreadPoolExecutor to download multiple blocks in parallel
    offsets = [offset + i*block_size for i in range(max_workers)]
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_block, o): o for o in offsets}
        
        all_done = False
        for future in as_completed(futures):
            result = future.result()
            if result is None:
                all_done = True
                continue
            
            block_offset, table = result
            
            # Initialize ParquetWriter on first block
            if first_block:
                writer = pq.ParquetWriter('rodent_inspection_data.parquet', table.schema, compression='snappy')
                first_block = False
            
            writer.write_table(table)
            print(f"Downloaded and wrote block at offset {block_offset}")
    if all_done:
        break
    offset += max_workers * block_size
    
# Closing file writer
if writer:
    writer.close()

print("All data downloaded and saved to Parquet.")

Downloaded and wrote block at offset 300000
Downloaded and wrote block at offset 200000
Downloaded and wrote block at offset 100000
Downloaded and wrote block at offset 0
Downloaded and wrote block at offset 700000
Downloaded and wrote block at offset 400000
Downloaded and wrote block at offset 600000
Downloaded and wrote block at offset 500000
Downloaded and wrote block at offset 1100000
Downloaded and wrote block at offset 1000000
Downloaded and wrote block at offset 800000
Downloaded and wrote block at offset 900000
Downloaded and wrote block at offset 1400000
Downloaded and wrote block at offset 1200000
Downloaded and wrote block at offset 1300000
Downloaded and wrote block at offset 1500000
Downloaded and wrote block at offset 1800000
Downloaded and wrote block at offset 1700000
Downloaded and wrote block at offset 1900000
Downloaded and wrote block at offset 1600000
Downloaded and wrote block at offset 2200000
Downloaded and wrote block at offset 2100000
Downloaded and wrote bloc